# Sprint13 TensorFlow

In [1]:
import tensorflow as tf

a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a,b)

sess = tf.Session()
output = sess.run(add)
print(output)
sess.close()

12


In [2]:
with tf.Session() as sess:
    outoput = sess.run(add)
    print('a',a)
    print(a.eval())
    print("add",add)
    print(output)

a Tensor("Const:0", shape=(), dtype=int32)
5
add Tensor("Add:0", shape=(), dtype=int32)
12


In [3]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n)

12


## TensorFlowにおける値の扱い方
>TensorFlowが値を扱う上で独自の概念として、placeholderとValiableがあります。



placeholderはデータフローグラフの構築時には値が決まっていないものに使います。最初は配列の形だけ定義しておいて後から値を入れて使う空箱のような存在です。学習ごとに違う値が入る入力データや正解データなどに用いられます。

In [4]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c,d)

sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
#セッションを実行する際に引数feed_dictを使い、placeholderに入れる値を辞書型で指定します。
#ここを書き換えることで異なる計算が可能になります。
print(output)

output = sess.run(add, feed_dict={c:20, d:32})
print(output)
#ミニバッチ学習を行うような場合を想定すると、　placeholder の必要性がわかります。

12
52


>《Valiable》
Valiableはplaceholderとは違い、データフローグラフの構築時にも値を持ち、更新を行うものに対して使います。学習するパラメータ（重み、バイアス）に用いられます。

>《constant》
確認になりますが、placeholderでもValiableでもないただの値は定数constantとして扱います。

In [5]:
add = a + b
output = sess.run(add)
print(output)

12


In [6]:
sess.close()

### セッションのインスタンス化から終了までに対して、with構文を使うことも可能です

In [7]:
'''
with tf.Session() as sess:
    sess.run() # ここに計算の実行コードを入れていく
'''

'\nwith tf.Session() as sess:\n    sess.run() # ここに計算の実行コードを入れていく\n'

## 論理回路
>簡単な題材として、ロジスティック回帰による論理回路の再現を行います。論理回路は2つの値を入力し、1つの値を出力する関数のようなものです。入力も出力も0か1のみで表され、入力される組み合わせによって出力する値が変わります。
ANDゲートは入力された2つの値が両方とも1だった場合、出力が1となり、それ以外の組み合わせでは0を出力します。

In [8]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]]) #(4, 2)
y_train = np.array([[0],[0],[0],[1]]) #(4, 1)

### データフローグラフの構築
>学習データをTensorFlowのデータフローグラフに入力するための placeholder を用意しましょう。placeholderはデータフローグラフを作成する段階では値が決まっていない、空箱のような存在でした。

In [9]:
'''
第一引数のtf.float32で行列要素の数値のデータ型を指定しています。
第二引数の[None,2]で行列の形を指定しています。
ここで定義されている2はデータの次元を表しています。Noneの部分はデータ数を表す部分です。
今回のANDゲートの場合のデータ数は[0,0],[0,1],[1,0],[1,1]の4つしかないので
Noneの部分を[4,2]としても問題はありません。
しかし、任意の数のデータを入れられるように、一般的にはNoneを使います。
'''
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

>重みとバイアスの Valiable を用意します。Valiableとして用意するということは、これらが学習により更新を行う値であることを示します。

In [10]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

>次にモデルの出力y（＝仮定関数）と目的関数を定義します。

In [11]:
y = tf.sigmoid(tf.matmul(x,W) + b)
#tf.matmul()はNumPyにおけるnp.dot()に相当する
#ベクトルの内積や、行列積を計算するためのメソッドです。

cross_entropy = tf.reduce_sum(-t * tf.log(y) 
                              - (1 - t) * tf.log(1 - y))

>ここまでで、入力のための空箱であるplaceholderと学習可能なValiableをメソッドで結ぶことができました。
学習を行うために、勾配降下法を用いてパラメータを最適化するためのコードを加えます。目的関数をGradientDescentOptimizerに渡します。

In [12]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
#GradientDescentOptimizer()は引数で学習率を指定しています。

>学習後の結果の正解が正しいかどうかの判定と正解率の計算もデータフローグラフとして定義できます。

In [13]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
"""
1行目で結果が正解かどうか判定しています。
一つ一つ見ていきましょう。
まずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。
返り値はBool値です。tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。
yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。

2行目は正解率を計算するためのコードです。
tf.reduce_mean()は多次元配列の各成分の平均を計算する関数です。
tf.cast()でBool値を0,1に変換しています。
つまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので
正解率を表していることになります。
"""

'\n1行目で結果が正解かどうか判定しています。\n一つ一つ見ていきましょう。\nまずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。\n返り値はBool値です。tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。\nyが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。\n\n2行目は正解率を計算するためのコードです。\ntf.reduce_mean()は多次元配列の各成分の平均を計算する関数です。\ntf.cast()でBool値を0,1に変換しています。\nつまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので\n正解率を表していることになります。\n'

## データを入力して計算

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
'''
まずセッションのインスタンスを作成します。
そして、tf.global_variables_initializer()によって
上で定義したtf.Variable()の値(重み・バイアス)を初期化します。
実行する際にはsess.run()を使います。
'''

'\nまずセッションのインスタンスを作成します。\nそして、tf.global_variables_initializer()によって\n上で定義したtf.Variable()の値(重み・バイアス)を初期化します。\n実行する際にはsess.run()を使います。\n'

In [15]:
#学習を行います。今回は1000回繰り返すことにします。

for epoch in range(1000):
    sess.run(train_step, feed_dict = {
        x:x_train,
        t:y_train})
    
    # 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict = {
                x:x_train,
                t:y_train})
        print('epoch: %d, Accuracy: %f'
              %(epoch, acc_val))
        
'''
2行目、sess.run()に上で定義したtrain_stepを入れることで、
勾配降下法による学習を行っています。
8行目はsess.run()にaccuracyを入れることで、正解率を計算しています。
計算結果がNumPy形式で返ってきているので、これをprintします。

形だけ定義していたplaceholderのxとtの中には値が何も入っていません。
placeholderに値を設定するためにsess.run()のパラメータでfeed_dictを指定します。
例えば、feed_dict={x:x_train,t:y_train})と書くことで
空箱だったxにx_trainの値が入り、tにy_trainの値が入ります。

表示結果を見てみると、正解率が100%でうまく学習できているように見えます。
'''

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


'\n2行目、sess.run()に上で定義したtrain_stepを入れることで、\n勾配降下法による学習を行っています。\n8行目はsess.run()にaccuracyを入れることで、正解率を計算しています。\n計算結果がNumPy形式で返ってきているので、これをprintします。\n\n形だけ定義していたplaceholderのxとtの中には値が何も入っていません。\nplaceholderに値を設定するためにsess.run()のパラメータでfeed_dictを指定します。\n例えば、feed_dict={x:x_train,t:y_train})と書くことで\n空箱だったxにx_trainの値が入り、tにy_trainの値が入ります。\n\n表示結果を見てみると、正解率が100%でうまく学習できているように見えます。\n'

>各サンプルの計算結果を確認してロジスティック回帰の実装を終わります。先ほどのaccuracyと同様、実行することで計算結果が返ってくるためprintします。

In [16]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict = {
    x:x_train,
    t:y_train})

#出力yの確認
prob = sess.run(y, feed_dict = {
    x:x_train,
    t:y_train})

print(classified)
print(prob)
'''
classifiedの結果は全てTrueで正しく学習されていることがわかります。
probは上からほぼ[0,0,0,1]となっています。
今回活性化関数に用いたのはシグモイド関数ですので出力yは確率として表示されています。
上から3つは1になる確率がほぼ0％、一番下の1つは93％程度の確率で1になるということになります。
'''

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9654632e-04]
 [4.9049824e-02]
 [4.9049824e-02]
 [9.3120384e-01]]


'\nclassifiedの結果は全てTrueで正しく学習されていることがわかります。\nprobは上からほぼ[0,0,0,1]となっています。\n今回活性化関数に用いたのはシグモイド関数ですので出力yは確率として表示されています。\n上から3つは1になる確率がほぼ0％、一番下の1つは93％程度の確率で1になるということになります。\n'

In [17]:
print('W:',sess.run(W))
print('b:',sess.run(b))

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


### 途中の値が見たい場合
>デバッグのために途中の値が見たい場合もあります。例えば、y = tf.sigmoid(tf.matmul(x, W) + b)のtf.matmul(x, W)の部分が見たいといったことを考えます。
出力yがsess.run(y, feed_dict={x:x_train, t:y_train})で見れたことと同様の行えば良いため、次のようなコードになります。

In [18]:
mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)
print(sess.run(mat, feed_dict = {
    x:x_train,
    t:y_train}))
sess.close()

#データフローグラフを構築し、それをsess.run()で実行するという流れはここでも同じです。

[[ 0.       ]
 [ 5.5699544]
 [ 5.5699544]
 [11.139909 ]]


# Sprint13 DLFlameWork 1

In [19]:
# x_train = np.array([[0,0],[0,1],[1,0],[1,1]]) #(4, 2)
# y_train = np.array([[0],[0],[0],[1]]) #(4, 1)

## 【問題1】スクラッチを振り返る
>ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

In [20]:
#重みの初期化が必要だった
#層ごとに使う関数とハイパーパラメーターを設定しなければならなかった
#ミニバッジとエポックでループする必要があった。
#クラス作成やクラス内関数を数多く作らなければならなかった

データセットの用意


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from numpy import linalg as LA
import copy
sns.set()
%matplotlib inline
import time
import copy

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
from sklearn.datasets import load_iris
iris = load_iris()
#print(iris)

X = pd.DataFrame(iris.data,columns=iris.feature_names) #(150, 4)

y = pd.DataFrame(iris.target,columns={'Species'}) #(150, 1)
#print(iris.target_names) #['setosa' 'versicolor' 'virginica']
df = pd.concat([X, y], axis=1) #(150, 5)
data = df[df['Species'] !=0] #(100, 5)

## 【問題2】スクラッチとTensorFlowの対応を考える
>先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [23]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 損失関数の出力をトレース対象とする
# loss_summary = tf.summary.scalar('loss', loss)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    
    #TENSOR BOARD
    tf.summary.scalar('cross_entropy', loss_op)
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter('data', graph = sess.graph)
    
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        
        summary_str = sess.run(summary_op, feed_dict={X: X_val, Y: y_val})
        summary_writer.add_summary(summary_str, epoch)
        
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    
    summary_writer.flush()
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 56.5591, val_loss : 26.5573, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 12.0300, val_loss : 25.6138, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 19.1267, val_loss : 10.8235, acc : 0.250, val_acc : 0.625
Epoch 3, loss : 6.6376, val_loss : 13.9818, acc : 0.750, val_acc : 0.375
Epoch 4, loss : 7.9972, val_loss : 5.9252, acc : 0.500, val_acc : 0.688
Epoch 5, loss : 6.1204, val_loss : 12.6583, acc : 0.750, val_acc : 0.375
Epoch 6, loss : 3.1816, val_loss : 3.5615, acc : 0.500, val_acc : 0.750
Epoch 7, loss : 2.1083, val_loss : 4.5454, acc : 0.750, val_acc : 0.562
Epoch 8, loss : 0.4695, val_loss : 2.3228, acc : 0.750, val_acc : 0.812
Epoch 9, loss : 0.1924, val_loss : 2.4479, acc : 0.750, val_acc : 0.750
test_acc : 0.700


In [24]:
! tensorboard --logdir=data --port=8088

W1206 16:53:36.378894 123145398910976 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 16:53:36.400886 123145398910976 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 16:53:36.425184 123145398910976 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 16:53:36.444333 123145398910976 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 16:53:36.474127 123145398910976 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or mor

^C


In [23]:
from keras import backend as K
K.clear_session()
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    # 1エポック内でのイテレーション回数を返す
    def __len__(self):
        return self._stop
    # 各イテレーションで使うバッチデータを返す
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    # イテレーションのカウント数をゼロにする
    def __iter__(self):
        self._counter = 0
        return self
    # 次のイテレーションで使うバッチデータを返す
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder(dtype="float", shape=[None, n_input])
Y = tf.placeholder(dtype="float", shape=[None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # he_normal()の初期化
    initializer = tf.initializers.he_uniform()
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.get_variable(name='W1', shape=[n_input, n_hidden1], initializer=initializer),
        'w2': tf.get_variable(name='W2', shape=[n_hidden1, n_hidden2], initializer=initializer),
        'w3': tf.get_variable(name='W3', shape=[n_hidden2, 1], initializer=initializer)
    }
    biases = {
        'b1': tf.get_variable(name='b1', shape=[n_hidden1], initializer=initializer),
        'b2': tf.get_variable(name='b2', shape=[n_hidden2], initializer=initializer),
        'b3': tf.get_variable(name='b3', shape=[1], initializer=initializer)
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    # summaryの設定
    
#     # 指定したディレクトリがあれば削除し、再作成
#     if tf.gfile.Exists(log_dir):
#         tf.gfile.DeleteRecursively(log_dir)
# 　　　　tf.gfile.MakeDirs(log_dir)
    
    tf.summary.scalar('cross_entropy', loss_op)
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter('data', graph=sess.graph)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        total_val_loss = 0
        total_val_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_loss += loss
            total_acc += acc
            total_val_loss += val_loss
            total_val_acc += val_acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_val_loss /= total_batch
        total_val_acc /= total_batch    
        #tf.summary.FileWriter('iris_sigmoid', sess.graph)
        summary_str = sess.run(summary_op, feed_dict={X: X_val, Y: y_val})
        summary_writer.add_summary(summary_str, epoch)
        print("Epoch {}, total_loss : {:.4f}, total_val_loss : {:.4f}, total_acc : {:.3f}, total_val_acc : {:.3f}".format(
            epoch, total_loss, total_val_loss, total_acc, total_val_acc))
    # tensorboardでスカラを表示させるために必要
    summary_writer.flush()  
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Using TensorFlow backend.


Epoch 0, total_loss : 2.9373, total_val_loss : 2.1309, total_acc : 0.407, total_val_acc : 0.446
Epoch 1, total_loss : 0.5168, total_val_loss : 0.4098, total_acc : 0.757, total_val_acc : 0.804
Epoch 2, total_loss : 0.4277, total_val_loss : 0.4271, total_acc : 0.729, total_val_acc : 0.732
Epoch 3, total_loss : 0.4903, total_val_loss : 0.3992, total_acc : 0.671, total_val_acc : 0.777
Epoch 4, total_loss : 0.3156, total_val_loss : 0.2443, total_acc : 0.857, total_val_acc : 0.982
Epoch 5, total_loss : 0.2110, total_val_loss : 0.1981, total_acc : 0.957, total_val_acc : 0.982
Epoch 6, total_loss : 0.2676, total_val_loss : 0.2115, total_acc : 0.900, total_val_acc : 0.929
Epoch 7, total_loss : 0.1916, total_val_loss : 0.1539, total_acc : 0.943, total_val_acc : 0.991
Epoch 8, total_loss : 0.1741, total_val_loss : 0.1451, total_acc : 0.957, total_val_acc : 0.991
Epoch 9, total_loss : 0.1870, total_val_loss : 0.1431, total_acc : 0.914, total_val_acc : 0.991
test_acc : 0.900


In [24]:
! tensorboard --logdir=data

W1206 17:19:05.507390 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 17:19:05.542107 123145340358656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 17:19:05.570373 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 17:19:05.591423 123145340358656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 17:19:05.618563 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or mor

W1206 17:19:06.169155 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 17:19:06.169493 123145340358656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 17:19:06.180128 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W1206 17:19:06.180448 123145340358656 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W1206 17:19:06.190329 123145340358656 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or mor

In [ ]:
#backwardの作成が不要。手間が半分。
#重みとバイアスの初期化をまとめて行っているため、見やすい。
#それぞれの段階で何を行っているのかが見やすい。
#計算が関数で用意されているので関数作成の手間が大幅に削減されている。
#ループ、ハイパーパラメーターは相変わらず手作業での実装が必要。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
>risデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。
2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [ ]:
#以下の2箇所は2クラス分類特有の処理です。

loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

In [104]:
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", 
               "PetalLengthCm", "PetalWidthCm"]]
y = df["Species"]

y = np.array(y)
X = np.array(X) #(150, 4)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis] 

"""
one_hot化が必要
"""
onehot = OneHotEncoder(sparse=False)
y = onehot.fit_transform(y) #one_hot(150, 3)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, 
                                    batch_size = batch_size)

def net_3_layers(x):
    # 重みとバイアスの宣言
    weights = {
        'w1':tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2':tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3':tf.Variable(tf.random_normal([n_hidden2, n_classes]))}
    biases = {'b1':tf.Variable(tf.random_normal([n_hidden1])),
              'b2':tf.Variable(tf.random_normal([n_hidden2])),
              'b3':tf.Variable(tf.random_normal([n_classes]))}
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']),biases['b1']) 
    #tf.matmul = @, dot積
    layer_1 = tf.nn.relu(layer_1) 
    #tf.nn.relu -> フィルタやバイアスを足しこんだ結果(Wx + b)のテンソルを利用する
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) #softmaxに変更
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output


# ネットワーク構造の読み込み                               
logits = net_3_layers(X)

'''
loss_op, correct_pred を多値分類用に変更
softmax
'''

# 目的関数

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                         (labels = Y, logits = logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果

correct_pred = tf.equal(tf.sign(Y - 0.5), 
                        tf.sign(tf.sigmoid(logits) - 0.5))
#tf.equal 渡された２つのベクトルが一致しているか否かを見る。返り値[True, False]。

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化 *大切
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #np.ceil 切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, 
                mini_batch_y) in enumerate(get_mini_batch_train):
            #enumerate()関数を使うと、
            #forループの中でリスト（配列）などのイテラブルオブジェクトの要素と同時に
            #インデックス番号（カウント、順番）を取得できる。
            sess.run(train_op, 
                     feed_dict = {X: mini_batch_x, 
                                  Y: mini_batch_y})
            #tf.placeholder()値は、sess.run()のfeed_dictに辞書型で指定
            loss, acc = sess.run([loss_op, accuracy],
                                 feed_dict = {X: mini_batch_x, 
                                              Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy],
                                     feed_dict = {X: X_val,
                                                  Y: y_val})
        print("Epoch{}, loss:{:.4f},\
        val_loss:{:.4f}, total_acc:{:.3f}, val_acc:{:.3f}"
              .format(epoch, loss, val_loss, total_acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict = {X: X_test,
                                               Y: y_test})
    print("test_acc:{:.3f}".format(test_acc))
    
    
#tensorboad で結果見てみる

/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/envs/py37/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Epoch0, loss:17.2233,        val_loss:6.1041, total_acc:0.506, val_acc:0.542
Epoch1, loss:2.1804,        val_loss:1.7385, total_acc:0.521, val_acc:0.667
Epoch2, loss:0.0020,        val_loss:0.6543, total_acc:0.667, val_acc:0.667
Epoch3, loss:0.0000,        val_loss:1.7449, total_acc:0.673, val_acc:0.667
Epoch4, loss:0.0003,        val_loss:2.3319, total_acc:0.683, val_acc:0.708
Epoch5, loss:0.0063,        val_loss:2.9026, total_acc:0.690, val_acc:0.736
Epoch6, loss:2.8167,        val_loss:5.8170, total_acc:0.693, val_acc:0.750
Epoch7, loss:0.0000,        val_loss:0.9056, total_acc:0.684, val_acc:0.694
Epoch8, loss:0.0000,        val_loss:2.8221, total_acc:0.678, val_acc:0.708
Epoch9, loss:0.0000,        val_loss:2.9801, total_acc:0.681, val_acc:0.708
test_acc:0.711


## 【問題4】House Pricesのモデルを作成
>回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。
目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。
分類問題と回帰問題の違いを考慮してください。

In [190]:
"""
初期化の順番が大事な時はinitialized_valueを使うこと。

"""
# データセットの読み込み
data = pd.read_csv('train.csv')
y = data['SalePrice'] #(1460,)
y = y.astype(np.int)[:, np.newaxis] #(1460, 1)
X = data.loc[:,['GrLivArea','YearBuilt']] #(1460, 2)
y = np.array(y).astype(np.float64)
X = np.array(X).astype(np.float64)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

#分類問題と回帰問題の違いを考慮してください。

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(y_test.dtype)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]

'''
n_classes = 3　を回帰問題にする時にどうする？ -> クラスではなく 1 で固定。
'''

# 計算グラフに渡す引数の形を決める
X = tf.placeholder(dtype = tf.float32, shape = [None, n_input])
Y = tf.placeholder(dtype = tf.float32, shape = [None, 1])


# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, 
                                    batch_size = batch_size)

def net_houseplice(x):
    # 重みとバイアスの宣言
    weights = {
        'w1':tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2':tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3':tf.Variable(tf.random_normal([n_hidden2, 1]))}
    biases = {'b1':tf.Variable(tf.random_normal([n_hidden1])),
              'b2':tf.Variable(tf.random_normal([n_hidden2])),
              'b3':tf.Variable(tf.random_normal([1]))}
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']),biases['b1']) 
    #tf.matmul = @, dot積
    layer_1 = tf.nn.relu(layer_1) 
    #tf.nn.relu -> フィルタやバイアスを足しこんだ結果(Wx + b)のテンソルを利用する
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output


# ネットワーク構造の読み込み                               
logits = net_houseplice(X) 


'''
loss_op, correct_pred を回帰問題用に変更
平均二乗誤差（MSE: Mean Squared Error）は回帰問題に使われる一般的な損失関数
回帰問題の一般的な評価指標は平均絶対誤差（MAE: Mean Absolute Error）

一般にラプラス分布は正規分布よりも裾野が広く、多くの外れ値が観測されます。
多くの外れ値が存在するデータの誤差を評価したい、
あるいは外れ値にあまり影響されない評価を行いたい場合、
RMSE より MAE のほうが優れた指標であるといえるでしょう。

他方、基本的な線形回帰(MLR, PLS)から deep learning まで、
最小化する関数には二乗誤差が使われていることも見逃せません。
'''

# 目的関数 MSE

loss_op = tf.reduce_mean(tf.square(logits - Y))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)




# 指標値計算
#精度、評価　RMSE
# RMSE = tf.sqrt(loss_op)
#tf.sqrt -> √

# variableの初期化 *大切
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #np.ceil 切り上げ
        total_loss = 0
        for i, (mini_batch_x, 
                mini_batch_y) in enumerate(get_mini_batch_train):
            #enumerate()関数を使うと、
            #forループの中でリスト（配列）などのイテラブルオブジェクトの要素と同時に
            #インデックス番号（カウント、順番）を取得できる。
            sess.run(train_op,
                     feed_dict = {X: mini_batch_x, 
                                  Y: mini_batch_y})
            #tf.placeholder()値は、sess.run()のfeed_dictに辞書型で指定
            loss = sess.run(loss_op,
                            feed_dict = {X: mini_batch_x, 
                                         Y: mini_batch_y})
            
            total_loss += loss
            
        
        total_loss /= total_batch
        val_loss = sess.run(loss_op,
                            feed_dict = {X: X_val,
                                         Y: y_val})
        print("Epoch{}, loss:{:.4f},\
        val_loss:{:.4f}".format(epoch, loss, val_loss))
    
    test_loss = sess.run(loss_op, feed_dict = {X: X_test,
                                               Y: y_test})
    print("test_loss:{:.3f}".format(test_loss))
    
#対数取ってない、正規化してないのでloss でかい
#tensorboad で結果見てみる

float64
Epoch0, loss:1204820224.0000,        val_loss:2498041600.0000
Epoch1, loss:1328785408.0000,        val_loss:2532409088.0000
Epoch2, loss:1269164800.0000,        val_loss:2514784768.0000
Epoch3, loss:1210956032.0000,        val_loss:2503635456.0000
Epoch4, loss:1212569472.0000,        val_loss:2493856256.0000
Epoch5, loss:1214144896.0000,        val_loss:2494605056.0000
Epoch6, loss:1229514624.0000,        val_loss:2499302912.0000
Epoch7, loss:1223481216.0000,        val_loss:2494694144.0000
Epoch8, loss:1234858496.0000,        val_loss:2495266304.0000
Epoch9, loss:1194202496.0000,        val_loss:2494798848.0000
test_loss:3967448064.000


## 【問題5】MNISTのモデルを作成
>ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。
3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。
スクラッチで実装したモデルの再現を目指してください。

In [25]:
#入力が画像である
# MNIST data set
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64
X_train = X_train.reshape(-1, 784) /255
X_test = X_test.reshape(-1, 784) /255

'''
ニューラルネットワークに入れる数字が大きいとうまく働かないことがあるので、
正規化した方が良い
'''


X_train, X_val, y_train_one_hot, y_val_one_hot = train_test_split(X_train, y_train_one_hot, test_size=0.2)


print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)


class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

(60000,)
(60000, 10)
float64
(48000, 784)
(12000, 784)


In [26]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder(dtype = tf.float32, shape = [None, n_input])
Y = tf.placeholder(dtype = tf.float32, shape = [None, n_classes])
Y_ = tf.placeholder(dtype = tf.float32, shape = [None, n_classes])


# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, 
                                    batch_size = batch_size)

def net_3_layers(x):
    # 重みとバイアスの宣言
    weights = {
        'w1':tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2':tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3':tf.Variable(tf.random_normal([n_hidden2, n_classes]))}
    biases = {'b1':tf.Variable(tf.random_normal([n_hidden1])),
              'b2':tf.Variable(tf.random_normal([n_hidden2])),
              'b3':tf.Variable(tf.random_normal([n_classes]))}
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']),biases['b1']) 
    #tf.matmul = @, dot積
    layer_1 = tf.nn.relu(layer_1) 
    #tf.nn.relu -> フィルタやバイアスを足しこんだ結果(Wx + b)のテンソルを利用する
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) #softmaxに変更
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output


# ネットワーク構造の読み込み                               
logits = net_3_layers(X)

'''
loss_op, correct_pred を多値分類用に変更
softmax
'''

# 目的関数

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                         (labels = Y, logits = logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果

correct_pred = tf.equal(tf.argmax(logits, 1),tf.argmax(Y, 1))
#tf.equal 渡された２つのベクトルが一致しているか否かを見る。返り値[True, False]。

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化 *大切
init = tf.global_variables_initializer()




# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #np.ceil 切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, 
                mini_batch_y) in enumerate(get_mini_batch_train):
            #enumerate()関数を使うと、
            #forループの中でリスト（配列）などのイテラブルオブジェクトの要素と同時に
            #インデックス番号（カウント、順番）を取得できる。
            sess.run(train_op, 
                     feed_dict = {X: mini_batch_x, 
                                  Y: mini_batch_y})
            #tf.placeholder()値は、sess.run()のfeed_dictに辞書型で指定
            loss, acc = sess.run([loss_op, accuracy],
                                 feed_dict = {X: mini_batch_x, 
                                              Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy],
                                     feed_dict = {X: X_val,
                                                  Y: y_val_one_hot})
        print("Epoch{}, loss:{:.4f},\
        val_loss:{:.4f}, total_acc:{:.3f}, val_acc:{:.3f}"
              .format(epoch, loss, val_loss, total_acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict = {X: X_test,
                                               Y: y_test_one_hot})
    print("test_acc:{:.3f}".format(test_acc))
    
    
#tensorboad で結果見てみる

Epoch0, loss:0.5086,        val_loss:0.9717, total_acc:0.712, val_acc:0.713
Epoch1, loss:0.5939,        val_loss:0.8216, total_acc:0.768, val_acc:0.796
Epoch2, loss:0.1736,        val_loss:0.5039, total_acc:0.850, val_acc:0.888
Epoch3, loss:0.0229,        val_loss:0.3571, total_acc:0.902, val_acc:0.913
Epoch4, loss:0.0084,        val_loss:0.3203, total_acc:0.919, val_acc:0.920
Epoch5, loss:0.0458,        val_loss:0.3891, total_acc:0.928, val_acc:0.913
Epoch6, loss:0.0135,        val_loss:0.3078, total_acc:0.932, val_acc:0.928
Epoch7, loss:0.0175,        val_loss:0.3005, total_acc:0.935, val_acc:0.929
Epoch8, loss:0.0048,        val_loss:0.3091, total_acc:0.938, val_acc:0.926
Epoch9, loss:0.0275,        val_loss:0.3227, total_acc:0.939, val_acc:0.928
test_acc:0.931


In [ ]:
# CNN ver
# https://qiita.com/yoyoyo_/items/7ecd9ef446d255c7d866 に回答あり

'''
今回は白黒画像ですからチャンネルは1つしかありませんが、
チャンネル方向の軸は用意しておく必要があります。
(n_samples, n_channels, height, width)のNCHW
または(n_samples, height, width, n_channels)のNHWC どちらかの形にしてください。
'''
#(n_samples, n_channels, height, width)のNCHW
X_train = X_train.reshape(48000, 1, 28, 28)
X_val = X_val.reshape(12000, 1, 28, 28)

print(X_train.shape) # (48000, 1, 28, 28)
print(X_val.shape) # (12000, 1, 28, 28)


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, 
                                    batch_size = batch_size)


def conv2d# CNN ver
# https://qiita.com/yoyoyo_/items/7ecd9ef446d255c7d866 に回答あり(x, F_si
          )



def conv_net(x, F_size = 3, 
           channel_in = 1, channel_out = 1,
           strides = 1):
    weights = {
        'w1':tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2':tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3':tf.Variable(tf.random_normal([n_hidden2, n_classes]))}
    biases = {'b1':tf.Variable(tf.random_normal([n_hidden1])),
              'b2':tf.Variable(tf.random_normal([n_hidden2])),
              'b3':tf.Variable(tf.random_normal([n_classes]))}
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']),biases['b1']) 
    #tf.matmul = @, dot積
    layer_1 = tf.nn.relu(layer_1) 
    #tf.nn.relu -> フィルタやバイアスを足しこんだ結果(Wx + b)のテンソルを利用する
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) #softmaxに変更
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output


# ネットワーク構造の読み込み                               
logits = conv_net(X)

'''
loss_op, correct_pred を多値分類用に変更
softmax
'''

# 目的関数

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                         (labels = Y, logits = logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果

correct_pred = tf.equal(tf.sign(Y - 0.5), 
                        tf.sign(tf.sigmoid(logits) - 0.5))
#tf.equal 渡された２つのベクトルが一致しているか否かを見る。返り値[True, False]。

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化 *大切
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #np.ceil 切り上げ
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, 
                mini_batch_y) in enumerate(get_mini_batch_train):
            #enumerate()関数を使うと、
            #forループの中でリスト（配列）などのイテラブルオブジェクトの要素と同時に
            #インデックス番号（カウント、順番）を取得できる。
            sess.run(train_op, 
                     feed_dict = {X: mini_batch_x, 
                                  Y: mini_batch_y})
            #tf.placeholder()値は、sess.run()のfeed_dictに辞書型で指定
            loss, acc = sess.run([loss_op, accuracy],
                                 feed_dict = {X: mini_batch_x, 
                                              Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy],
                                     feed_dict = {X: X_val,
                                                  Y: y_val})
        print("Epoch{}, loss:{:.4f},\
        val_loss:{:.4f}, total_acc:{:.3f}, val_acc:{:.3f}"
              .format(epoch, loss, val_loss, total_acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict = {X: X_test,
                                               Y: y_test})
    print("test_acc:{:.3f}".format(test_acc))
    
    
#tensorboad で結果見てみる